In [ ]:
# Imports
from IOHexperimenter import IOH_function, IOH_logger
from IOH_function_wrapper import IOH_function_logger
import input_reader
import parameters
import datetime
from shutil import make_archive
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# Create a dictionary that contains algorithms and splitpoints / targets
data = input_reader.create_dict()

In [ ]:
class Switch:
    """ Class to implement the switch between algorithms.

    Parameters:
    ---------------
    func : object, callable
        The function to be optimized.

    budget : int
        The budget that is available for function evaluations.

    data : dict
        A dictionary with tau as key, algorithm as value


    Attibutes:
    ---------------
    None

    """

    def __init__(self, func, budget, data):
        self.func = func
        self.budget = budget * func.number_of_variables
        self.data = data

    def __call__(self):
        """ Runs the switch routine. 
        
        Returns:
        -----------
        best_so_far_variables : array
                The best found solution.

        best_so_far_fvaluet: float
               The fitness value for the best found solution.
        """

        # Initialize parameters
        params = parameters.init_params(self.budget)

        # Iterate through dictionary
        for tau, alg in data.items():
            algorithm = alg(self.func)
            # Set algorithm parameters based on parameters object
            algorithm.set_params(params)

            # Define stopping criteria
            func = self.func
            budget = self.budget

            def stopping_criteria():
                return (func.best_so_far_precision) <= tau or (
                    func.evaluations > budget)

            algorithm.set_stopping_criteria(stopping_criteria)

            # Run algorithm and extract parameters
            algorithm.run()
            params = algorithm.get_params(params)

            if self.func.evaluations >= budget:
                break

        # Return best point and fitness value found by last run algorithm
        return (algorithm.func.best_so_far_variables, algorithm.func.best_so_far_precision, params) 

In [ ]:
# Experiment routine
path = "/Users/dschroeder/Documents/Master Computer Science/Master_Thesis/code_and_data/experiments/"
now = datetime.datetime.now()
experiment_ID = str(now.day) + "-" + str(now.month) + "-" + str(
    now.strftime('%H')) + str(now.strftime('%M')) + str(now.strftime('%S'))
dimensions = [5]
instances = [1, 2, 3, 4, 5]
func_ids = [21]
exp_budget = 20000
runs = 5
switch_combination = "BFGS-CMAES"
settings = "regression_C_and_sigma"
experiment_name = path + switch_combination + "/experiment" + experiment_ID + "_" + settings

for func_id in func_ids:
    for dim in dimensions:
        for inst in instances:
            f = IOH_function_logger(func_id, dim, inst, suite='BBOB')
            log = IOH_logger(experiment_name, "run",
                             switch_combination + settings, "switch-routine")
            f.add_logger(log)
            for run in range(0, runs):
                if run > 0:
                    f.reset()
                    f.x_hist = []
                    f.f_hist = []
                switch = Switch(f, exp_budget, data)
                x_opt, prec, params = switch()
                print(f' run {run} dim{dim} f{func_id} inst{inst} complete, precision: {prec}')
            f.clear_logger()

make_archive(experiment_name, "zip", experiment_name)

In [ ]:
switch.func.evaluations, switch.func.best_so_far_variables

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

x_hist_cmaes = params.internal_dict['cmaes_x_hist']
x_hist_bfgs = params.internal_dict['a1_x_hist']

x1_coords_cmaes = []
x2_coords_cmaes = []
#for i in range(0, 10000):
for i in range(0, int(1 * len(x_hist_cmaes))):
    x1_coords_cmaes.append(x_hist_cmaes[i][0])
    x2_coords_cmaes.append(x_hist_cmaes[i][1])

x1_coords_bfgs = []
x2_coords_bfgs = []
for i in range(0, int(1 * len(x_hist_bfgs))):
    x1_coords_bfgs.append(x_hist_bfgs[i][0])
    x2_coords_bfgs.append(x_hist_bfgs[i][1])

x_opt_bfgs = params.internal_dict['BFGS_bestpoint']
x_opt_cmaes = params.internal_dict['cmaes_x_opt']

In [ ]:
# Contour plot
#[1.94863206 2.92102641]
x_lb = -0.9
x_ub = -0.8
y_lb = -1.3
y_ub = -1.2
sample_size = 50
x_step = (x_ub - x_lb) / sample_size
y_step = (y_ub - y_lb) / sample_size
x_coords = np.arange(x_lb, x_ub, x_step)
y_coords = np.arange(y_lb, y_ub, y_step)
X, Y = np.meshgrid(x_coords[0:sample_size], y_coords[0:sample_size])

In [ ]:
Zlog = np.zeros((sample_size, sample_size))
for i in range(0, sample_size):
    for j in range(0, sample_size):
        Zlog[i][j] = np.log10(f((X[i][j], Y[i][j])) - f.get_target())

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

fig, ax = plt.subplots()  # Create a figure containing a single axes.
ax.plot(x1_coords_cmaes, x2_coords_cmaes, 'ro', label = 'population(CMA-ES)', color='green', alpha=0.1)
ax.plot(x1_coords_bfgs, x2_coords_bfgs, 'ro', label = 'population(BFGS)', color='red')
ax.plot(x_opt [0], x_opt[1], 'ro', label = 'solution(switch)', color='blue')
ax.plot(x_opt_bfgs[0], x_opt_bfgs[1], 'ro', label = 'solution(BFGS)', color='orange')
ax.set_xlabel('x1')  # Add an x-label to the axes.
ax.set_ylabel('x2')  # Add a y-label to the axes.
ax.legend()  # Add a legend.
ax.axis('equal')
ax.set_xlim(left=x_lb, right=x_ub)
ax.set_ylim(bottom=y_lb, top=y_ub)
levels = np.arange(-5, 5, 0.1)
cs = ax.contourf(X, Y, Zlog, cmap = 'viridis', levels=levels)
#fig.colorbar(cs).set_label('log10 target precision')
#ax.clabel(cs, inline=1, fontsize=8)
plt.savefig(switch_combination + settings + 'ellipsoid_H_as_C_afterswitch_multip_sigma_zoom2')

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
evals = params.internal_dict['evalcount']
stepsizes = params.internal_dict['stepsizes']
splitpoint_evals = params.internal_dict['evals_splitpoint']
fig, ax = plt.subplots()  # Create a figure containing a single axes.
ax.plot(evals, stepsizes, label = 'step size(switch)')
ax.set_xlabel('evaluations')  # Add an x-label to the axes.
ax.set_ylabel('step size')  # Add a y-label to the axes.
ax.legend()  # Add a legend.
plt.vlines(splitpoint_evals, 0, max(stepsizes)+0.2, colors='red', linestyles='dotted', label='split point')
#ax.set_xlim(left=0, right=800, emit=True, auto=False)
ax.set_ylim(bottom=0, top=0.01)
plt.savefig(switch_combination + settings + '_stepsize_zoom2')

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
evals = params.internal_dict['evalcount']
matrix_norm = params.internal_dict['matrix_norm']
splitpoint_evals = params.internal_dict['evals_splitpoint']
fig, ax = plt.subplots()  # Create a figure containing a single axes.
ax.plot(evals, matrix_norm, label = 'matrix norm(switch)')
ax.set_xlabel('evaluations')  # Add an x-label to the axes.
ax.set_ylabel('matrix norm')  # Add a y-label to the axes.
ax.legend()  # Add a legend.
plt.vlines(splitpoint_evals, 0, max(matrix_norm)+1, colors='red', linestyles='dotted', label='split point')
#ax.set_xlim(left=0, right=800, emit=True, auto=False)
ax.set_ylim(bottom=0, top=1)
plt.savefig(switch_combination + settings + '_matrixnorm_zoom1')

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
fig, ax = plt.subplots()  # Create a figure containing a single axes.
ax.plot(evals, stepsizes, label = 'stepsizes(switch)')
ax.set_xlabel('evaluations')  # Add an x-label to the axes.
ax.set_ylabel('step size')  # Add a y-label to the axes.
#ax.set_xlim(left=, right=5000, emit=True, auto=False)
ax.legend()  # Add a legend.
plt.vlines(1659, 0, 5, colors='red', linestyles='dotted', label='split point')
#plt.savefig('sigma-change-switch-bfgs-cmaes-zoom')